In [1]:
import pytest
import numpy as np
from thewalrus import mtl, lmtl
from thewalrus.quantum import Qmat, Xmat
from thewalrus.reference import rspm, rpmp, mtl as mtl_symb
from thewalrus.random import random_covariance
from scipy.special import factorial2
from scipy.stats import unitary_group

In [6]:
n = 4
for perfect in rpmp(range(1, 2 * n + 1)):
    print(perfect)

((1, 6), (2, 7), (3, 8), (5, 4))
((1, 6), (2, 7), (3, 4), (5, 8))
((1, 6), (2, 3), (7, 8), (5, 4))
((1, 6), (2, 3), (7, 4), (5, 8))
((1, 2), (6, 7), (3, 8), (5, 4))
((1, 2), (6, 7), (3, 4), (5, 8))
((1, 2), (6, 3), (7, 8), (5, 4))
((1, 2), (6, 3), (7, 4), (5, 8))
((1, 6), (2, 8), (4, 7), (5, 3))
((1, 6), (2, 8), (4, 3), (5, 7))
((1, 6), (2, 4), (8, 7), (5, 3))
((1, 6), (2, 4), (8, 3), (5, 7))
((1, 2), (6, 8), (4, 7), (5, 3))
((1, 2), (6, 8), (4, 3), (5, 7))
((1, 2), (6, 4), (8, 7), (5, 3))
((1, 2), (6, 4), (8, 3), (5, 7))
((1, 7), (3, 6), (2, 8), (5, 4))
((1, 7), (3, 6), (2, 4), (5, 8))
((1, 7), (3, 2), (6, 8), (5, 4))
((1, 7), (3, 2), (6, 4), (5, 8))
((1, 3), (7, 6), (2, 8), (5, 4))
((1, 3), (7, 6), (2, 4), (5, 8))
((1, 3), (7, 2), (6, 8), (5, 4))
((1, 3), (7, 2), (6, 4), (5, 8))
((1, 8), (4, 6), (2, 7), (5, 3))
((1, 8), (4, 6), (2, 3), (5, 7))
((1, 8), (4, 2), (6, 7), (5, 3))
((1, 8), (4, 2), (6, 3), (5, 7))
((1, 4), (8, 6), (2, 7), (5, 3))
((1, 4), (8, 6), (2, 3), (5, 7))
((1, 4), (

In [5]:
n = 3
test = True
for perfect in rpmp(range(1, 2 * n + 1)):
    last = perfect[0][1]  # starting point
    reduced_last = last - n if last > n else last
    # different mode in every tuple
    if reduced_last == 1:
        test = False

    for i in perfect[1:]:
        reduced = i[0] - n if i[0] > n else i[0], i[1] - n if i[1] > n else i[1]
        # different mode in every tuple
        if reduced[0] == reduced[1]:
            test = False
        # consecutive tuple contain the same mode
        if reduced_last not in reduced:
            test = False

        last = i[0] if reduced[1] == reduced_last else i[1]
        reduced_last = last - n if last > n else last

    # last mode most coincide with the first one
    if reduced_last != 1:
        test = False

print(test)        

True


In [8]:
def rpmp_alternating_walk(perfect, n):
    last = perfect[0][1]  # starting point
    reduced_last = last - n if last > n else last
    # different mode in every tuple
    if reduced_last == 1:
        return False

    for i in perfect[1:]:
        reduced = i[0] - n if i[0] > n else i[0], i[1] - n if i[1] > n else i[1]
        # different mode in every tuple
        if reduced[0] == reduced[1]:
            return False
        # consecutive tuple contain the same mode
        if reduced_last not in reduced:
            return False

        last = i[0] if reduced[1] == reduced_last else i[1]
        reduced_last = last - n if last > n else last

    # last mode most coincide with the first one
    if reduced_last != 1:
        return False
    
    return True

In [10]:
n = 4
test = True
for perfect in rpmp(range(1, 2 * n + 1)):
    test = rpmp_alternating_walk(perfect, n)
    if not test:
        break

print(test)

True


In [22]:
def rspm_alternating_walk(perfect, n):
    # First tuple should be a loop
    if perfect[0][0] != perfect[0][1]:
        return False
    
    last = perfect[0][1]  # starting point
    reduced_last = last - n if last > n else last

    for i in perfect[1:-1]:
        reduced = i[0] - n if i[0] > n else i[0], i[1] - n if i[1] > n else i[1]
        # different mode in every tuple
        if reduced[0] == reduced[1]:
            return False
        # consecutive tuple contain the same mode
        if reduced_last not in reduced:
            print(f'reduced_last={reduced_last} and reduced = {reduced}')
            return False

        last = i[0] if reduced[1] == reduced_last else i[1]
        reduced_last = last - n if last > n else last

    #consecutive tuple contain the same mode
    if perfect[-1][0] not in (last, reduced_last, last+n):
        return False
    
    # last mode most be a loop
    if perfect[-1][0] != perfect[-1][1]:
        return False
    
    return True

In [23]:
n = 4
test = True
for perfect in rspm(range(1, 2 * n + 1)):
    test = rpmp_alternating_walk(perfect, n) if len(perfect)==n else rspm_alternating_walk(perfect, n)
    print(f'{test} : {perfect}')
    if not test:
        break

print(f'final test : {test}')

True : ((1, 6), (2, 7), (3, 8), (5, 4))
True : ((6, 6), (2, 7), (3, 8), (5, 4), (1, 1))
True : ((7, 7), (3, 8), (5, 4), (1, 6), (2, 2))
True : ((8, 8), (5, 4), (1, 6), (2, 7), (3, 3))
reduced_last=4 and reduced = (1, 2)
False : ((4, 4), (1, 6), (2, 7), (3, 8), (5, 5))
final test : False
